In [ ]:
!pip install openai

     |████████████████████████████████| 42 kB 788 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 162 kB 6.8 MB/s 
  Created wheel for openai: filename=openai-0.18.1-py3-none-any.whl size=53168 sha256=5e4c73b83250c0af62e555f0b794864b95f78047b86b10f58983ca5ca087c3af
  Stored in directory: /root/.cache/pip/wheels/5a/bf/24/fcdc9d2b81f9c7e565bb2036ec9f7cc930056b829895b3bf48
Successfully built openai


In [ ]:
import os
import openai
import copy
import scipy.stats

In [ ]:
import pandas

### Ordinals

In [ ]:
cars = """class: vhigh, high, med, low (ORDINAL)
buying: vhigh, high, med, low (ORDINAL)
maint: vhigh, high, med, low (ORDINAL)
doors: 2, 3, 4, 5, more (ORDINAL)
persons: 2, 4, more (ORDINAL)
lug_boot: small, med, big (ORDINAL)
safety: low, med, high (ORDINAL)"""

nursery = """class: not_recom, recommend, very_recom, priority, spec_prior (ORDINAL)
parents: usual, pretentious, great_pret (ORDINAL)
has_nurs: proper, less_proper, improper, critical, very_crit (ORDINAL)
form: complete, completed, incomplete, foster (ORDINAL)
children: 1, 2, 3, more (ORDINAL)
housing: convenient, less_conv, critical (ORDINAL)
social: non-prob, slightly_prob, problematic (ORDINAL)
health: recommended, priority, not_recom (ORDINAL)"""

breast_cancer = """age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99 (ORDINAL)
menopause: lt40, ge40, premeno (ORDINAL)
tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59 (ORDINAL)
inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39 (ORDINAL)
breast-quad: left-up, left-low, right-up, right-low, central (NON-ORDINAL)"""

post_operative = """L-CORE: high, mid, low (ORDINAL)
L-SURF: high, mid, low (ORDINAL)
L-O2: excellent, good, fair, poor (ORDINAL)
L-BP: high, mid, low (ORDINAL)
SURF-STBL: stable, mod-stable, unstable (ORDINAL)
CORE-STBL: stable, mod-stable, unstable (ORDINAL)
BP-STBL: stable, mod-stable, unstable (ORDINAL)
Decision: I (patient sent to Intensive Care Unit), A (patient sent to general hospital floor), S (patient prepared to go home) (ORDINAL)"""

soy_bean = """date: april,may,june,july,august,september,october (ORDINAL)
precip: lt-norm,norm,gt-norm  (ORDINAL)
temp: lt-norm,norm,gt-norm  (ORDINAL)
crop-hist: diff-lst-year,same-lst-yr,same-lst-two-yrs,same-lst-sev-yrs   (ORDINAL)
area-damaged: scattered,low-areas,upper-areas,whole-field (NON-ORDINAL)
severity: minor,pot-severe,severe   (ORDINAL)
seed-tmt: none,fungicide,other   (NON-ORDINAL)
germination: 90-100%,80-89%,lt-80%   (ORDINAL)
leafspots-halo: absent,yellow-halos,no-yellow-halos (NON-ORDINAL)
leafspots-marg: w-s-marg,no-w-s-marg,dna (NON-ORDINAL)
leafspot-size: lt-1/8,gt-1/8,dna (NON-ORDINAL)
stem-cankers: absent,below-soil,above-soil,above-sec-nde (NON-ORDINAL)
canker-lesion: dna,brown,dk-brown-blk,tan (NON-ORDINAL)
mycelium: absent,present (NON-ORDINAL)
fruit-pods: norm,diseased,few-present,dna  (NON-ORDINAL)
fruit spots: absent,colored,brown-w/blk-specks,distort,dna (NON-ORDINAL)
roots: norm,rotted,galls-cysts (NON-ORDINAL)"""

mushroom = """cap-shape: bell,conical,convex,flat,knobbed,sunken (NON-ORDINAL)
cap-surface: fibrous,grooves,scaly,smoot (NON-ORDINAL)
cap-color: brown,buff,cinnamon,gray,green, pink,purple,red,white,yellow (NON-ORDINAL)
odor: almond,anise,creosote,fishy,foul, musty, none,pungent,spicy (NON-ORDINAL)
gill-attachment: attached,descending,free,notched (NON-ORDINAL)
gill-spacing: close,crowded,distant (NON-ORDINAL)
gill-color: black,brown,buff,chocolate,gray, green,orange,pink,purple,red, white,yellow (NON-ORDINAL)
stalk-root: bulbous,club,cup,equal, rhizomorphs,rooted,missing (NON-ORDINAL)
stalk-surface-above-ring: fibrous,scaly,silky,smooth (NON-ORDINAL)
stalk-surface-below-ring: fibrous,scaly,silky,smooth (NON-ORDINAL)
stalk-color-above-ring: brown,buff,cinnamon,gray,orange, pink,red,white,yellow (NON-ORDINAL)
stalk-color-below-ring: brown,buff,cinnamon,gray,orange, pink,red,white,yellow (NON-ORDINAL)
veil-color: brown,orange,white,yellow (NON-ORDINAL)
ring-number: none,one,two (ORDINAL)
ring-type: cobwebby,evanescent,flaring,large, none,pendant,sheathing,zone (NON-ORDINAL)
spore-print-color: black,brown,buff,chocolate,green, orange,purple,white,yellow (NON-ORDINAL)
population: abundant,clustered,numerous, scattered,several,solitary (ORDINAL)
habitat: grasses,leaves,meadows,paths, urban,waste,woods (NON-ORDINAL)"""

In [ ]:
datasets = {"cars": cars, "nursery": nursery, "breast_cancer": breast_cancer, "post_operative": post_operative, "soy_bean": soy_bean, "mushroom": mushroom}

In [ ]:
datasets_dict = {}

for dataset_name in datasets:
  datasets_dict[dataset_name] = {}
  dataset = datasets[dataset_name].split("\n")
  for variable in dataset:
    var_name = variable.split(':')[0]
    datasets_dict[dataset_name][var_name] = {}
    datasets_dict[dataset_name][var_name]["classes"] = variable.split(':')[1].split('(')[0].replace(" ", "").split(",")
    datasets_dict[dataset_name][var_name]["target"] = variable.split(':')[1].split('(')[1].replace(")", "")


In [ ]:
datasets_dict

{'breast_cancer': {'age': {'classes': ['10-19',
    '20-29',
    '30-39',
    '40-49',
    '50-59',
    '60-69',
    '70-79',
    '80-89',
    '90-99'],
   'target': 'ORDINAL'},
  'breast-quad': {'classes': ['left-up',
    'left-low',
    'right-up',
    'right-low',
    'central'],
   'target': 'NON-ORDINAL'},
  'inv-nodes': {'classes': ['0-2',
    '3-5',
    '6-8',
    '9-11',
    '12-14',
    '15-17',
    '18-20',
    '21-23',
    '24-26',
    '27-29',
    '30-32',
    '33-35',
    '36-39'],
   'target': 'ORDINAL'},
  'menopause': {'classes': ['lt40', 'ge40', 'premeno'], 'target': 'ORDINAL'},
  'tumor-size': {'classes': ['0-4',
    '5-9',
    '10-14',
    '15-19',
    '20-24',
    '25-29',
    '30-34',
    '35-39',
    '40-44',
    '45-49',
    '50-54',
    '55-59'],
   'target': 'ORDINAL'}},
 'cars': {'buying': {'classes': ['vhigh', 'high', 'med', 'low'],
   'target': 'ORDINAL'},
  'class': {'classes': ['vhigh', 'high', 'med', 'low'], 'target': 'ORDINAL'},
  'doors': {'classes': ['

In [ ]:
prompt_1_pred = 'Is "house" higher than "apartment"? Yes\nIs "apartment" higher than "house"? No\nIs "red" higher than "blue"? No\nIs "blue" higher than "red"? No\nIs "old" higher than "young"? Yes\nIs "young" higher than "old"? No\nIs "totally agree" higher than "agree"? Yes\nIs "agree" higher than "totally agree"? No\nIs "New York" higher than "Chicago"? No\nIs "Chicago" higher than "New York"? No\nIs "Heavy rain" higher than "Showers"? Yes\nIs "Showers" higher than "Heavy rain"? No'
prompt_2_pred = 'Is (low, medium, high) an ordinal? Yes\nIs (door, window, wheel) an ordinal? No' 


In [ ]:
openai.api_key = "COMPLETE"

### Prompt 1

In [ ]:
results_prompt1 = {}

for dataset in datasets_dict:
  print(f"Dataset: {dataset}")
  results_prompt1[dataset] = {}

  for attribute in datasets_dict[dataset]:
    print(f"Attribute: {attribute}")

    ranks = {}

    for rank, elem in enumerate(datasets_dict[dataset][attribute]["classes"]):
      ranks[elem] = {}
      ranks[elem]["real"] = rank
      ranks[elem]["predicted"] = 0

    for elem_i in datasets_dict[dataset][attribute]["classes"]:

      for elem_j in [elem for elem in datasets_dict[dataset][attribute]["classes"] if elem != elem_i]:

        prompt = f'{prompt_1_pred}\nIs "{elem_i}" higher than "{elem_j}"?'
        response = openai.Completion.create(engine="text-davinci-002", prompt=prompt, temperature=0, max_tokens=6, stop=[".", ",", "\n"])
        text = response["choices"][0]["text"]
        print(prompt)
        print(text)

        if text.replace("\n", "").replace(" ", "")[0:3] in ["Yes", "yes"]:
          ranks[elem_i]["predicted"] += 1
          ranks[elem_j]["predicted"] -= 1

        elif text.replace("\n", "").replace(" ", "")[0:2] in ["No", "no"]:
          ranks[elem_i]["predicted"] -= 1
          ranks[elem_j]["predicted"] += 1

    results_prompt1[dataset][attribute] = ranks

Streaming output truncated to the last 5000 lines.
Is "young" higher than "old"? No
Is "totally agree" higher than "agree"? Yes
Is "agree" higher than "totally agree"? No
Is "New York" higher than "Chicago"? No
Is "Chicago" higher than "New York"? No
Is "Heavy rain" higher than "Showers"? Yes
Is "Showers" higher than "Heavy rain"? No
Is "brown" higher than "red"?
 No
Is "house" higher than "apartment"? Yes
Is "apartment" higher than "house"? No
Is "red" higher than "blue"? No
Is "blue" higher than "red"? No
Is "old" higher than "young"? Yes
Is "young" higher than "old"? No
Is "totally agree" higher than "agree"? Yes
Is "agree" higher than "totally agree"? No
Is "New York" higher than "Chicago"? No
Is "Chicago" higher than "New York"? No
Is "Heavy rain" higher than "Showers"? Yes
Is "Showers" higher than "Heavy rain"? No
Is "brown" higher than "white"?
 No
Is "house" higher than "apartment"? Yes
Is "apartment" higher than "house"? No
Is "red" higher than "blue"? No
Is "blue" higher than

In [ ]:
results_prompt1

{'breast_cancer': {'age': {'10-19': {'predicted': -8, 'real': 0},
   '20-29': {'predicted': -8, 'real': 1},
   '30-39': {'predicted': -10, 'real': 2},
   '40-49': {'predicted': -6, 'real': 3},
   '50-59': {'predicted': -4, 'real': 4},
   '60-69': {'predicted': 0, 'real': 5},
   '70-79': {'predicted': 8, 'real': 6},
   '80-89': {'predicted': 12, 'real': 7},
   '90-99': {'predicted': 16, 'real': 8}},
  'breast-quad': {'central': {'predicted': 0, 'real': 4},
   'left-low': {'predicted': 0, 'real': 1},
   'left-up': {'predicted': 0, 'real': 0},
   'right-low': {'predicted': 0, 'real': 3},
   'right-up': {'predicted': 0, 'real': 2}},
  'inv-nodes': {'0-2': {'predicted': -6, 'real': 0},
   '12-14': {'predicted': -4, 'real': 4},
   '15-17': {'predicted': -10, 'real': 5},
   '18-20': {'predicted': -4, 'real': 6},
   '21-23': {'predicted': 2, 'real': 7},
   '24-26': {'predicted': 10, 'real': 8},
   '27-29': {'predicted': 2, 'real': 9},
   '3-5': {'predicted': -8, 'real': 1},
   '30-32': {'predi

In [ ]:
def bubble_sort_custom(nums):
    # We set swapped to True so the loop looks runs at least once
    swapped = True
    while swapped:
        swapped = False
        for i in range(len(nums) - 1):
            if nums[i][1]["real"] > nums[i + 1][1]["real"]:
                # Swap the elements
                nums[i], nums[i + 1] = nums[i + 1], nums[i]
                # Set the flag to True so we'll loop again
                swapped = True
    return nums

In [ ]:
evaluation_prompt1 = {}

for dataset in results_prompt1:
  print(f"Dataset: {dataset}")
  evaluation_prompt1[dataset] = {}
  for attribute in results_prompt1[dataset]:
    print(f"Attribute: {attribute}")
    evaluation_prompt1[dataset][attribute] = {}

    results = bubble_sort_custom(list(results_prompt1[dataset][attribute].items()))
    print(results)

    real_rank = [elem[1]["real"] for elem in results]
    predicted_rank = [elem[1]["predicted"] for elem in results]

    abs_spear_corr = abs(scipy.stats.spearmanr(real_rank, predicted_rank)[0])
    print(abs_spear_corr)

    evaluation_prompt1[dataset][attribute]["abs_corr"] = abs_spear_corr

    if abs_spear_corr > 0.75:
      prediction = "ORDINAL"
    else:
      prediction = "NON-ORDINAL"

    evaluation_prompt1[dataset][attribute]["prediction"] = prediction

    if prediction == datasets_dict[dataset][attribute]["target"]:
      evaluation_prompt1[dataset][attribute]["match"] = 1
    else: 
      evaluation_prompt1[dataset][attribute]["match"] = 0

Dataset: breast_cancer
Attribute: age
[('10-19', {'predicted': -8, 'real': 0}), ('20-29', {'predicted': -8, 'real': 1}), ('30-39', {'predicted': -10, 'real': 2}), ('40-49', {'predicted': -6, 'real': 3}), ('50-59', {'predicted': -4, 'real': 4}), ('60-69', {'predicted': 0, 'real': 5}), ('70-79', {'predicted': 8, 'real': 6}), ('80-89', {'predicted': 12, 'real': 7}), ('90-99', {'predicted': 16, 'real': 8})]
0.9456149718939671
Attribute: breast-quad
[('left-up', {'predicted': 0, 'real': 0}), ('left-low', {'predicted': 0, 'real': 1}), ('right-up', {'predicted': 0, 'real': 2}), ('right-low', {'predicted': 0, 'real': 3}), ('central', {'predicted': 0, 'real': 4})]
nan
Attribute: inv-nodes
[('0-2', {'predicted': -6, 'real': 0}), ('3-5', {'predicted': -8, 'real': 1}), ('6-8', {'predicted': -2, 'real': 2}), ('9-11', {'predicted': -6, 'real': 3}), ('12-14', {'predicted': -4, 'real': 4}), ('15-17', {'predicted': -10, 'real': 5}), ('18-20', {'predicted': -4, 'real': 6}), ('21-23', {'predicted': 2, 'r

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [ ]:
evaluation_prompt1

{'breast_cancer': {'age': {'abs_corr': 0.9456149718939671,
   'match': 1,
   'prediction': 'ORDINAL'},
  'breast-quad': {'abs_corr': nan, 'match': 1, 'prediction': 'NON-ORDINAL'},
  'inv-nodes': {'abs_corr': 0.7972482047700905,
   'match': 1,
   'prediction': 'ORDINAL'},
  'menopause': {'abs_corr': nan, 'match': 0, 'prediction': 'NON-ORDINAL'},
  'tumor-size': {'abs_corr': 0.9033517474653049,
   'match': 1,
   'prediction': 'ORDINAL'}},
 'cars': {'buying': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'},
  'class': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'},
  'doors': {'abs_corr': 0.7, 'match': 0, 'prediction': 'NON-ORDINAL'},
  'lug_boot': {'abs_corr': 0.8660254037844387,
   'match': 1,
   'prediction': 'ORDINAL'},
  'maint': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'},
  'persons': {'abs_corr': 0.5, 'match': 0, 'prediction': 'NON-ORDINAL'},
  'safety': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'}},
 'mushroom': {'cap-color': {'abs_corr': nan,


In [ ]:
evaluation_prompt1 = {'breast_cancer': {'age': {'abs_corr': 0.9456149718939671,
   'match': 1,
   'prediction': 'ORDINAL'},
  'breast-quad': {'abs_corr': float("nan"), 'match': 1, 'prediction': 'NON-ORDINAL'},
  'inv-nodes': {'abs_corr': 0.7972482047700905,
   'match': 1,
   'prediction': 'ORDINAL'},
  'menopause': {'abs_corr': float("nan"), 'match': 0, 'prediction': 'NON-ORDINAL'},
  'tumor-size': {'abs_corr': 0.9033517474653049,
   'match': 1,
   'prediction': 'ORDINAL'}},
 'cars': {'buying': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'},
  'class': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'},
  'doors': {'abs_corr': 0.7, 'match': 0, 'prediction': 'NON-ORDINAL'},
  'lug_boot': {'abs_corr': 0.8660254037844387,
   'match': 1,
   'prediction': 'ORDINAL'},
  'maint': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'},
  'persons': {'abs_corr': 0.5, 'match': 0, 'prediction': 'NON-ORDINAL'},
  'safety': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'}},
 'mushroom': {'cap-color': {'abs_corr': float("nan"),
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'cap-shape': {'abs_corr': 0.08571428571428573,
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'cap-surface': {'abs_corr': float("nan"), 'match': 1, 'prediction': 'NON-ORDINAL'},
  'gill-attachment': {'abs_corr': float("nan"),
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'gill-color': {'abs_corr': float("nan"), 'match': 1, 'prediction': 'NON-ORDINAL'},
  'gill-spacing': {'abs_corr': 1.0, 'match': 0, 'prediction': 'ORDINAL'},
  'habitat': {'abs_corr': 0.9274260335029679,
   'match': 0,
   'prediction': 'ORDINAL'},
  'odor': {'abs_corr': 0.2797989667130676,
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'population': {'abs_corr': 0.14285714285714288,
   'match': 0,
   'prediction': 'NON-ORDINAL'},
  'ring-number': {'abs_corr': 0.8660254037844387,
   'match': 1,
   'prediction': 'ORDINAL'},
  'ring-type': {'abs_corr': 0.38324040381224284,
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'spore-print-color': {'abs_corr': float("nan"),
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'stalk-color-above-ring': {'abs_corr': float("nan"),
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'stalk-color-below-ring': {'abs_corr': float("nan"),
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'stalk-root': {'abs_corr': 0.4504687313477795,
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'stalk-surface-above-ring': {'abs_corr': float("nan"),
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'stalk-surface-below-ring': {'abs_corr': float("nan"),
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'veil-color': {'abs_corr': float("nan"), 'match': 1, 'prediction': 'NON-ORDINAL'}},
 'nursery': {'children': {'abs_corr': 0.7378647873726218,
   'match': 0,
   'prediction': 'NON-ORDINAL'},
  'class': {'abs_corr': 0.7826237921249264,
   'match': 1,
   'prediction': 'ORDINAL'},
  'form': {'abs_corr': 0.19999999999999998,
   'match': 0,
   'prediction': 'NON-ORDINAL'},
  'has_nurs': {'abs_corr': 0.7, 'match': 0, 'prediction': 'NON-ORDINAL'},
  'health': {'abs_corr': 0.5, 'match': 0, 'prediction': 'NON-ORDINAL'},
  'housing': {'abs_corr': 0.5, 'match': 0, 'prediction': 'NON-ORDINAL'},
  'parents': {'abs_corr': 0.0, 'match': 0, 'prediction': 'NON-ORDINAL'},
  'social': {'abs_corr': 0.5, 'match': 0, 'prediction': 'NON-ORDINAL'}},
 'post_operative': {'BP-STBL': {'abs_corr': float("nan"),
   'match': 0,
   'prediction': 'NON-ORDINAL'},
  'CORE-STBL': {'abs_corr': float("nan"), 'match': 0, 'prediction': 'NON-ORDINAL'},
  'Decision': {'abs_corr': float("nan"), 'match': 0, 'prediction': 'NON-ORDINAL'},
  'L-BP': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'},
  'L-CORE': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'},
  'L-O2': {'abs_corr': 0.632455532033676,
   'match': 0,
   'prediction': 'NON-ORDINAL'},
  'L-SURF': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'},
  'SURF-STBL': {'abs_corr': float("nan"), 'match': 0, 'prediction': 'NON-ORDINAL'}},
 'soy_bean': {'area-damaged': {'abs_corr': 0.7378647873726218,
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'canker-lesion': {'abs_corr': 0.316227766016838,
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'crop-hist': {'abs_corr': 0.316227766016838,
   'match': 0,
   'prediction': 'NON-ORDINAL'},
  'date': {'abs_corr': 0.7273929674533081,
   'match': 0,
   'prediction': 'NON-ORDINAL'},
  'fruit spots': {'abs_corr': 0.10540925533894596,
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'fruit-pods': {'abs_corr': 0.39999999999999997,
   'match': 1,
   'prediction': 'NON-ORDINAL'},
  'germination': {'abs_corr': 0.8660254037844387,
   'match': 1,
   'prediction': 'ORDINAL'},
  'leafspot-size': {'abs_corr': float("nan"), 'match': 1, 'prediction': 'NON-ORDINAL'},
  'leafspots-halo': {'abs_corr': 0.8660254037844387,
   'match': 0,
   'prediction': 'ORDINAL'},
  'leafspots-marg': {'abs_corr': float("nan"), 'match': 1, 'prediction': 'NON-ORDINAL'},
  'mycelium': {'abs_corr': 0.9999999999999999,
   'match': 0,
   'prediction': 'ORDINAL'},
  'precip': {'abs_corr': float("nan"), 'match': 0, 'prediction': 'NON-ORDINAL'},
  'roots': {'abs_corr': 1.0, 'match': 0, 'prediction': 'ORDINAL'},
  'seed-tmt': {'abs_corr': 0.5, 'match': 1, 'prediction': 'NON-ORDINAL'},
  'severity': {'abs_corr': 1.0, 'match': 1, 'prediction': 'ORDINAL'},
  'stem-cankers': {'abs_corr': 0.7999999999999999,
   'match': 0,
   'prediction': 'ORDINAL'},
  'temp': {'abs_corr': float("nan"), 'match': 0, 'prediction': 'NON-ORDINAL'}}}

In [ ]:
total_len = 0
total_match = 0
latex_dict_list = []

for dataset in evaluation_prompt1:
  latex_dict = {}
  dataset_match = 0
  print(dataset)
  latex_dict["dataset"] = dataset
  latex_dict["experiment"] = "ordinal"
  latex_dict["prompt"] = "prompt1"
  total_len += len(evaluation_prompt1[dataset])
  for attribute in evaluation_prompt1[dataset]:
    total_match += evaluation_prompt1[dataset][attribute]["match"]
    dataset_match += evaluation_prompt1[dataset][attribute]["match"]
  print(f"Hits: {dataset_match}")
  latex_dict["hits"] = dataset_match
  print(f"Failures: {len(evaluation_prompt1[dataset])-dataset_match}")
  latex_dict["failures"] = len(evaluation_prompt1[dataset])-dataset_match
  print(f"Dataset accuracy: {dataset_match / len(evaluation_prompt1[dataset])}")
  latex_dict["accuracy"] = dataset_match / len(evaluation_prompt1[dataset])
  latex_dict_list.append(latex_dict)
pandas.DataFrame(latex_dict_list).to_csv("prompt1_ordinal.csv")

print(f"Accuracy: {total_match / total_len}")

breast_cancer
Hits: 4
Failures: 1
Dataset accuracy: 0.8
cars
Hits: 5
Failures: 2
Dataset accuracy: 0.7142857142857143
mushroom
Hits: 15
Failures: 3
Dataset accuracy: 0.8333333333333334
nursery
Hits: 1
Failures: 7
Dataset accuracy: 0.125
post_operative
Hits: 3
Failures: 5
Dataset accuracy: 0.375
soy_bean
Hits: 9
Failures: 8
Dataset accuracy: 0.5294117647058824
Accuracy: 0.5873015873015873


### Prompt 2

In [ ]:
results_prompt2 = {}

for dataset in datasets_dict:
  print(f"Dataset: {dataset}")
  results_prompt2[dataset] = {}

  for attribute in datasets_dict[dataset]:
    print(f"Attribute: {attribute}")
    results_prompt2[dataset][attribute] = {}

    results_prompt2[dataset][attribute]["real"] = datasets_dict[dataset][attribute]["target"]

    prompt = f'{prompt_2_pred}\nIs {tuple(datasets_dict[dataset][attribute]["classes"])} an ordinal?'
    print(prompt)
    response = openai.Completion.create(engine="text-davinci-002", prompt=prompt, temperature=0, max_tokens=6, stop=[".", ",", "\n"])
    text = response["choices"][0]["text"]

    if text.replace("\n", "").replace(" ", "")[0:3] in ["Yes", "yes"]:
        results_prompt2[dataset][attribute]["predicted"] = "ORDINAL"
    elif text.replace("\n", "").replace(" ", "")[0:2] in ["No", "no"]:
        results_prompt2[dataset][attribute]["predicted"] = "NON-ORDINAL"
            

Dataset: cars
Attribute: class
Is (low, medium, high) an ordinal? Yes
Is (door, window, wheel) an ordinal? No
Is ('vhigh', 'high', 'med', 'low') an ordinal?
Attribute: buying
Is (low, medium, high) an ordinal? Yes
Is (door, window, wheel) an ordinal? No
Is ('vhigh', 'high', 'med', 'low') an ordinal?
Attribute: maint
Is (low, medium, high) an ordinal? Yes
Is (door, window, wheel) an ordinal? No
Is ('vhigh', 'high', 'med', 'low') an ordinal?
Attribute: doors
Is (low, medium, high) an ordinal? Yes
Is (door, window, wheel) an ordinal? No
Is ('2', '3', '4', '5', 'more') an ordinal?
Attribute: persons
Is (low, medium, high) an ordinal? Yes
Is (door, window, wheel) an ordinal? No
Is ('2', '4', 'more') an ordinal?
Attribute: lug_boot
Is (low, medium, high) an ordinal? Yes
Is (door, window, wheel) an ordinal? No
Is ('small', 'med', 'big') an ordinal?
Attribute: safety
Is (low, medium, high) an ordinal? Yes
Is (door, window, wheel) an ordinal? No
Is ('low', 'med', 'high') an ordinal?
Dataset: nu

In [ ]:
results_prompt2

{'breast_cancer': {'age': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'breast-quad': {'predicted': 'ORDINAL', 'real': 'NON-ORDINAL'},
  'inv-nodes': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'menopause': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'tumor-size': {'predicted': 'ORDINAL', 'real': 'ORDINAL'}},
 'cars': {'buying': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'class': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'doors': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'lug_boot': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'maint': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'persons': {'predicted': 'NON-ORDINAL', 'real': 'ORDINAL'},
  'safety': {'predicted': 'ORDINAL', 'real': 'ORDINAL'}},
 'mushroom': {'cap-color': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'cap-shape': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'cap-surface': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'gill-attachment': {'predicted': 'ORDINAL', 'real

In [ ]:
results_prompt2 = {'breast_cancer': {'age': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'breast-quad': {'predicted': 'ORDINAL', 'real': 'NON-ORDINAL'},
  'inv-nodes': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'menopause': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'tumor-size': {'predicted': 'ORDINAL', 'real': 'ORDINAL'}},
 'cars': {'buying': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'class': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'doors': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'lug_boot': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'maint': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'persons': {'predicted': 'NON-ORDINAL', 'real': 'ORDINAL'},
  'safety': {'predicted': 'ORDINAL', 'real': 'ORDINAL'}},
 'mushroom': {'cap-color': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'cap-shape': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'cap-surface': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'gill-attachment': {'predicted': 'ORDINAL', 'real': 'NON-ORDINAL'},
  'gill-color': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'gill-spacing': {'predicted': 'ORDINAL', 'real': 'NON-ORDINAL'},
  'habitat': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'odor': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'population': {'predicted': 'NON-ORDINAL', 'real': 'ORDINAL'},
  'ring-number': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'ring-type': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'spore-print-color': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'stalk-color-above-ring': {'predicted': 'NON-ORDINAL',
   'real': 'NON-ORDINAL'},
  'stalk-color-below-ring': {'predicted': 'NON-ORDINAL',
   'real': 'NON-ORDINAL'},
  'stalk-root': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'stalk-surface-above-ring': {'predicted': 'NON-ORDINAL',
   'real': 'NON-ORDINAL'},
  'stalk-surface-below-ring': {'predicted': 'NON-ORDINAL',
   'real': 'NON-ORDINAL'},
  'veil-color': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'}},
 'nursery': {'children': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'class': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'form': {'predicted': 'NON-ORDINAL', 'real': 'ORDINAL'},
  'has_nurs': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'health': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'housing': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'parents': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'social': {'predicted': 'ORDINAL', 'real': 'ORDINAL'}},
 'post_operative': {'BP-STBL': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'CORE-STBL': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'Decision': {'predicted': 'NON-ORDINAL',
   'real': 'patient sent to Intensive Care Unit, A '},
  'L-BP': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'L-CORE': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'L-O2': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'L-SURF': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'SURF-STBL': {'predicted': 'ORDINAL', 'real': 'ORDINAL'}},
 'soy_bean': {'area-damaged': {'predicted': 'ORDINAL', 'real': 'NON-ORDINAL'},
  'canker-lesion': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'crop-hist': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'date': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'fruit spots': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'fruit-pods': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'germination': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'leafspot-size': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'leafspots-halo': {'predicted': 'ORDINAL', 'real': 'NON-ORDINAL'},
  'leafspots-marg': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'mycelium': {'predicted': 'ORDINAL', 'real': 'NON-ORDINAL'},
  'precip': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'roots': {'predicted': 'ORDINAL', 'real': 'NON-ORDINAL'},
  'seed-tmt': {'predicted': 'NON-ORDINAL', 'real': 'NON-ORDINAL'},
  'severity': {'predicted': 'ORDINAL', 'real': 'ORDINAL'},
  'stem-cankers': {'predicted': 'ORDINAL', 'real': 'NON-ORDINAL'},
  'temp': {'predicted': 'ORDINAL', 'real': 'ORDINAL'}}}

In [ ]:
total_len = 0
total_match = 0
latex_dict_list = []

for dataset in results_prompt2:
  latex_dict = {}
  dataset_match = 0
  print(dataset)
  latex_dict["dataset"] = dataset
  latex_dict["experiment"] = "ordinal"
  latex_dict["prompt"] = "prompt2"
  total_len += len(results_prompt2[dataset])
  for attribute in results_prompt2[dataset]:
    if results_prompt2[dataset][attribute]["real"] == results_prompt2[dataset][attribute]["predicted"]:
      total_match += 1
      dataset_match += 1

  print(f"Hits: {dataset_match}")
  latex_dict["hits"] = dataset_match
  print(f"Failures: {len(results_prompt2[dataset]) - dataset_match}")
  latex_dict["failures"] = len(results_prompt2[dataset])-dataset_match
  print(f"Dataset accuracy: {dataset_match / len(results_prompt2[dataset])}")
  latex_dict["accuracy"] = dataset_match / len(results_prompt2[dataset])
  latex_dict_list.append(latex_dict)

pandas.DataFrame(latex_dict_list).to_csv("prompt2_ordinal.csv")
print(f"Accuracy: {total_match / total_len}")

breast_cancer
Hits: 4
Failures: 1
Dataset accuracy: 0.8
cars
Hits: 6
Failures: 1
Dataset accuracy: 0.8571428571428571
mushroom
Hits: 15
Failures: 3
Dataset accuracy: 0.8333333333333334
nursery
Hits: 7
Failures: 1
Dataset accuracy: 0.875
post_operative
Hits: 7
Failures: 1
Dataset accuracy: 0.875
soy_bean
Hits: 12
Failures: 5
Dataset accuracy: 0.7058823529411765
Accuracy: 0.8095238095238095
